## Pre-processing and building RAG Retrieval Prototype

#### What this script does:
- Loads processed text files
- Prepares documents for embedding by dividing into chunks 
- Creates embeddings using TF-IDF
- Saves vector index in 04_models/vector_index/
- Loads TF-IDF retriever from rag_componets/retriever.py
- Tests retriever with some basic automotive-focused questions

In [5]:
# Import libraries

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
import importlib.util
import sys
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import joblib

### Preprocessing pipeline

1. Load reports and divide into chunks

In [6]:
# Define paths where processed text files are found 
data_path = "../../01_data/rag_automotive_tech/processed"
papers_path = os.path.join(data_path, "automotive_papers") # Added journal abstracts file
patents_path = os.path.join(data_path, "automotive_tech_patents") # Added patent file
reports_path = os.path.join(data_path, "tech_reports")
startups_path = os.path.join(data_path, "startups")  # Added startups files

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

# Function for chunking documents from folders
def load_and_chunk_documents(folder_path, doc_type):
    """Load and chunk documents from a specific folder"""
    chunks = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            try:
                loader = TextLoader(file_path, encoding='utf-8')
                documents = loader.load()
                
                for doc in documents:
                    doc.metadata.update({
                        'source': filename,
                        'doc_type': doc_type,
                        'file_path': file_path
                    })
                
                doc_chunks = text_splitter.split_documents(documents)
                chunks.extend(doc_chunks)
                print(f"Loaded {len(doc_chunks)} chunks from {filename}")
                
            except Exception as e:
                print(f"Error loading {filename}: {e}")
    
    return chunks

# Load and chunk all documents
print("Loading research papers...")
papers_chunks = load_and_chunk_documents(papers_path, "research_paper")

print("Loading patents data...")
patents_chunks = load_and_chunk_documents(patents_path, "patents_data")

print("Loading tech reports...")
reports_chunks = load_and_chunk_documents(reports_path, "tech_report")

print("Loading startups data...")
startups_chunks = load_and_chunk_documents(startups_path, "startups")

# Combine all chunks
all_chunks = papers_chunks + patents_chunks + reports_chunks + startups_chunks
print(f"\nSummary:")
print(f"- Research papers: {len(papers_chunks)} chunks")
print(f"- Patents data: {len(patents_chunks)} chunks")
print(f"- Tech reports: {len(reports_chunks)} chunks")
print(f"- Startups data: {len(startups_chunks)} chunks")
print(f"Total chunks created: {len(all_chunks)}")

Loading research papers...
Loaded 61 chunks from enhanced_drift_aware_computer_vision_achitecture_for_autonomous_driving.txt
Loaded 102 chunks from Gen_AI_in_automotive_applications_challenges_and_opportunities_with_a_case_study_on_in-vehicle_experience.txt
Loaded 120 chunks from leveraging_vision_language_models_for_visual_grounding_and_analysis_of_automative_UI.txt
Loaded 11408 chunks from automotive_papers_processed.txt
Loaded 69 chunks from automating_automative_software_development_a_synergy_of_generative_AI_and_formal_methods.txt
Loaded 137 chunks from automotive-software-and-electronics-2030-full-report.txt
Loaded 102 chunks from AI_agents_in_engineering_design_a_multiagent_framework_for_aesthetic_and_aerodynamic_car_design.txt
Loaded 87 chunks from a_benchmark_framework_for_AI_models_in_automative_aerodynamics.txt
Loaded 227 chunks from generative_AI_for_autonomous_driving_a_review.txt
Loaded 46 chunks from Embedded_acoustic_intelligence_for_automotive_systems.txt
Loaded 107 ch

2. Create embeddings using FAISS and save Vector Index

Switched from TF-IDF = Bag-of-words, no semantic understanding to 
Word2Vec/Transformers = Semantic understanding, contextual meaning and FAISS (Chroma DB had issues, always corrupted/damaged - see below) 

Model: BAAI/bge-small-en
State-of-the-art for retrieval
Slightly larger but excellent

Used FastEmbed for good quality and loe memory (10-50x faster than regular sentence transformers)

In [15]:
import shutil

# NUCLEAR CLEANUP - Remove everything
vector_index_path = "../../04_models/vector_index"

if os.path.exists(vector_index_path):
    print(f"⚠️ Removing existing directory: {vector_index_path}")
    shutil.rmtree(vector_index_path)
    print("✓ Removed")

# Create fresh
os.makedirs(vector_index_path, exist_ok=True)
print(f"✓ Created fresh directory: {vector_index_path}")

⚠️ Removing existing directory: ../../04_models/vector_index
✓ Removed
✓ Created fresh directory: ../../04_models/vector_index


In [16]:
# ===== ULTRA-SIMPLE CHROMADB CREATION =====

import os
import pickle
from fastembed import TextEmbedding
import chromadb
import numpy as np

print(f"🚀 Creating embeddings for {len(all_chunks)} chunks")

# 1. EXTRACT TEXTS
texts = [chunk.page_content for chunk in all_chunks]

# 2. SETUP PATH (CORRECT!)
vector_index_path = "../../04_models/vector_index"

# 3. CLEANUP
import shutil
if os.path.exists(vector_index_path):
    print(f"🧹 Cleaning {vector_index_path}")
    shutil.rmtree(vector_index_path)
os.makedirs(vector_index_path, exist_ok=True)
print(f"📁 Fresh directory created")

# 4. CREATE EMBEDDINGS
print("🔮 Loading embedding model...")
model = TextEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

print("✨ Creating embeddings...")
embeddings = list(model.embed(texts))
print(f"✓ Created {len(embeddings)} embeddings")

# 5. INIT CHROMADB (ULTRA SIMPLE)
print("\n🛠️ Initializing ChromaDB...")

# Method A: Try simplest approach first
try:
    client = chromadb.PersistentClient(path=vector_index_path)
    print("✓ Client created")
    
    # Delete if exists
    try:
        client.delete_collection(name="documents")
        print("✓ Cleaned old collection")
    except:
        pass
    
    # Create collection
    collection = client.create_collection(name="documents")
    print(f"✓ Collection created: {collection.name}")
    
except Exception as e:
    print(f"❌ Method A failed: {e}")
    
    # Method B: Try with minimal settings
    print("Trying alternative method...")
    try:
        from chromadb.config import Settings
        
        client = chromadb.PersistentClient(
            path=vector_index_path,
            settings=Settings(
                anonymized_telemetry=False,
                chroma_server_grpc_port=None,
                chroma_server_http_port=None,
                chroma_server_ssl_enabled=False
            )
        )
        
        # Force delete if exists
        collections = client.list_collections()
        for col in collections:
            if col.name == "documents":
                client.delete_collection(name="documents")
        
        collection = client.create_collection(name="documents")
        print(f"✓ Collection created (Method B): {collection.name}")
        
    except Exception as e2:
        print(f"❌ Both methods failed: {e2}")
        print("Trying in-memory approach...")
        
        # Method C: In-memory as last resort
        client = chromadb.Client()
        collection = client.create_collection(name="documents")
        print("✓ Using in-memory collection (will not persist)")

# 6. ADD DOCUMENTS
print("\n📦 Adding documents...")
batch_size = 50
total_chunks = len(texts)

for i in range(0, total_chunks, batch_size):
    end_idx = min(i + batch_size, total_chunks)
    
    batch_texts = texts[i:end_idx]
    batch_embeddings = [emb.tolist() for emb in embeddings[i:end_idx]]
    
    batch_metadatas = []
    batch_ids = []
    
    for j in range(len(batch_texts)):
        chunk_idx = i + j
        chunk = all_chunks[chunk_idx]
        
        batch_metadatas.append({
            "source": str(chunk.metadata.get("source", "unknown")),
            "doc_type": str(chunk.metadata.get("doc_type", "document")),
            "chunk_index": chunk_idx
        })
        batch_ids.append(f"chunk_{chunk_idx}")
    
    try:
        collection.add(
            embeddings=batch_embeddings,
            documents=batch_texts,
            metadatas=batch_metadatas,
            ids=batch_ids
        )
        
        if (i // batch_size) % 20 == 0 or end_idx == total_chunks:
            print(f"  ✅ Added {end_idx}/{total_chunks} chunks")
            
    except Exception as e:
        print(f"  ⚠️ Batch {i}-{end_idx} failed: {e}")
        # Try without embeddings
        try:
            collection.add(
                documents=batch_texts,
                metadatas=batch_metadatas,
                ids=batch_ids
            )
            print(f"  ✅ Added without embeddings")
        except Exception as e2:
            print(f"  ❌ Complete failure: {e2}")

# 7. SAVE METADATA
print("\n💾 Saving metadata...")
try:
    chunks_metadata = []
    for i, chunk in enumerate(all_chunks):
        chunks_metadata.append({
            "page_content": chunk.page_content,
            "metadata": chunk.metadata,
            "embedding_index": i
        })
    
    metadata_path = os.path.join(vector_index_path, "chunks_metadata.pkl")
    with open(metadata_path, "wb") as f:
        pickle.dump(chunks_metadata, f)
    
    print(f"✓ Metadata saved: {metadata_path}")
    
except Exception as e:
    print(f"⚠️ Metadata save failed: {e}")

# 8. VERIFY
print("\n" + "="*50)
print("✅ FINAL VERIFICATION")
print("="*50)

print(f"\n📊 Stats:")
print(f"Total chunks: {len(texts)}")
print(f"Embeddings created: {len(embeddings)}")

try:
    count = collection.count()
    print(f"ChromaDB collection count: {count}")
    
    if count > 0:
        # Test query
        print("\n🧪 Test query: 'automotive'")
        results = collection.query(
            query_texts=["automotive"],
            n_results=2
        )
        
        if results and results['documents']:
            print(f"✓ Query successful! Found {len(results['documents'][0])} results")
            print(f"First result preview: {results['documents'][0][0][:100]}...")
        else:
            print("⚠️ Query returned no results")
    else:
        print("❌ Collection is empty!")
        
except Exception as e:
    print(f"❌ Verification failed: {e}")

# 9. CHECK DIRECTORY
print(f"\n📁 Directory contents of {vector_index_path}:")
try:
    files = os.listdir(vector_index_path)
    if files:
        for file in files:
            size = os.path.getsize(os.path.join(vector_index_path, file))
            print(f"  - {file} ({size:,} bytes)")
    else:
        print("  (empty)")
except Exception as e:
    print(f"  Could not list: {e}")

print("\n" + "="*50)
print("🎉 PROCESS COMPLETE!")
print("="*50)

🚀 Creating embeddings for 18717 chunks
🧹 Cleaning ../../04_models/vector_index
📁 Fresh directory created
🔮 Loading embedding model...
✨ Creating embeddings...
✓ Created 18717 embeddings

🛠️ Initializing ChromaDB...
❌ Method A failed: Database error: error returned from database: (code: 1) no such table: tenants
Trying alternative method...
❌ Both methods failed: An instance of Chroma already exists for ../../04_models/vector_index with different settings
Trying in-memory approach...
✓ Using in-memory collection (will not persist)

📦 Adding documents...
  ✅ Added 50/18717 chunks
  ✅ Added 1050/18717 chunks
  ✅ Added 2050/18717 chunks
  ✅ Added 3050/18717 chunks
  ✅ Added 4050/18717 chunks
  ✅ Added 5050/18717 chunks
  ✅ Added 6050/18717 chunks
  ✅ Added 7050/18717 chunks
  ✅ Added 8050/18717 chunks
  ✅ Added 9050/18717 chunks
  ✅ Added 10050/18717 chunks
  ✅ Added 11050/18717 chunks
  ✅ Added 12050/18717 chunks
  ✅ Added 13050/18717 chunks
  ✅ Added 14050/18717 chunks
  ✅ Added 15050/18

/Users/manueltimowolf/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:25<00:00, 3.21MiB/s]


✓ Query successful! Found 2 results
First result preview: RESEARCH PAPER #919:
  Title: Mapping the Landscape of Romanian Automotive Research: A Bibliometric ...

📁 Directory contents of ../../04_models/vector_index:
  - chunks_metadata.pkl (11,772,358 bytes)

🎉 PROCESS COMPLETE!


In [17]:
!pip install --upgrade chromadb==0.4.22 --force-reinstall

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Using cached chromadb-0.4.22-py3-none-any.whl.metadata (7.3 kB)
  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached fastapi-0.123.5-py3-none-any.whl.metadata (30 kB)
  Using cached uvicorn-0.38.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_x86_64.whl.metadata (60 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached onnxruntime-1.19.2-cp39-cp39-macosx_11_0_universal2.whl.metadata (4.5 kB)
  Using cached opentelemetry_api-1.39.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.39.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.60b0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemet

FAISS is battle-tested, reliable, and won't have database corruption issues. Let's implement this step-by-step:

In [18]:
# Run this cell in your notebook
!pip install faiss-cpu langchain -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
# ===== RELIABLE FAISS EMBEDDING SOLUTION =====
import os
import pickle
import numpy as np
from fastembed import TextEmbedding
import faiss
from datetime import datetime

print(f"🚀 RELIABLE FAISS - Processing {len(all_chunks)} chunks")

# Extract texts
texts = [chunk.page_content for chunk in all_chunks]
metadatas = [chunk.metadata for chunk in all_chunks]

# Setup paths
vector_index_path = "../../04_models/vector_index"
os.makedirs(vector_index_path, exist_ok=True)

# 1. Create embeddings
print("Loading fast embedding model...")
model = TextEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

print(f"Creating embeddings for {len(texts)} chunks...")
embeddings = list(model.embed(texts))
print(f"✓ Created {len(embeddings)} embeddings")

# Convert to numpy array
embeddings_array = np.array([emb.tolist() for emb in embeddings]).astype('float32')
print(f"Embedding shape: {embeddings_array.shape}")

# 2. Create FAISS index
print("\n🔧 Creating FAISS index...")
dimension = embeddings_array.shape[1]

# Create index (L2 distance - smaller is better)
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_array)

print(f"✓ FAISS index created with {index.ntotal} vectors")

# 3. Save everything
print("\n💾 Saving to disk...")

# Save FAISS index
faiss_index_path = os.path.join(vector_index_path, "faiss_index.bin")
faiss.write_index(index, faiss_index_path)
print(f"✓ FAISS index saved: {faiss_index_path}")

# Save texts
texts_path = os.path.join(vector_index_path, "texts.pkl")
with open(texts_path, "wb") as f:
    pickle.dump(texts, f)
print(f"✓ Texts saved: {texts_path} ({len(texts)} texts)")

# Save metadata
metadata_path = os.path.join(vector_index_path, "metadata.pkl")
with open(metadata_path, "wb") as f:
    pickle.dump(metadatas, f)
print(f"✓ Metadata saved: {metadata_path} ({len(metadatas)} entries)")

# Save embeddings for reference
embeddings_path = os.path.join(vector_index_path, "embeddings.npy")
np.save(embeddings_path, embeddings_array)
print(f"✓ Embeddings saved: {embeddings_path}")

# 4. Create index info
index_info = {
    "total_chunks": len(texts),
    "embedding_dim": dimension,
    "created_at": str(datetime.now()),
    "model": "sentence-transformers/all-MiniLM-L6-v2",
    "index_type": "FAISS IndexFlatL2"
}

info_path = os.path.join(vector_index_path, "index_info.json")
import json
with open(info_path, "w") as f:
    json.dump(index_info, f, indent=2)
print(f"✓ Index info saved: {info_path}")

# 5. VERIFICATION
print("\n" + "="*50)
print("✅ FAISS INDEX CREATION COMPLETE")
print("="*50)

print(f"\n📊 STATS:")
print(f"Total chunks: {len(texts)}")
print(f"Embedding dimension: {dimension}")
print(f"FAISS index size: {index.ntotal} vectors")

# Test query
print("\n🧪 TEST QUERY:")
test_query = "automotive technology"
print(f"Query: '{test_query}'")

# Create embedding for query
query_embedding = np.array(list(model.embed([test_query]))[0].tolist()).astype('float32').reshape(1, -1)

# Search
k = 3
distances, indices = index.search(query_embedding, k)

print(f"Found {len(indices[0])} results:")
for i, (idx, distance) in enumerate(zip(indices[0], distances[0])):
    if idx != -1:  # FAISS returns -1 if not enough results
        preview = texts[idx][:100] + "..." if len(texts[idx]) > 100 else texts[idx]
        doc_type = metadatas[idx].get('doc_type', 'unknown')
        print(f"\n  Result {i+1} (distance: {distance:.4f}):")
        print(f"    Type: {doc_type}")
        print(f"    Preview: {preview}")

# 6. Directory listing
print(f"\n📁 Directory contents of {vector_index_path}:")
for file in os.listdir(vector_index_path):
    file_path = os.path.join(vector_index_path, file)
    size = os.path.getsize(file_path)
    print(f"  - {file} ({size:,} bytes)")

print("\n" + "="*50)
print("🎉 RELIABLE VECTOR STORE READY!")
print("="*50)

🚀 RELIABLE FAISS - Processing 18717 chunks
Loading fast embedding model...
Creating embeddings for 18717 chunks...
✓ Created 18717 embeddings
Embedding shape: (18717, 384)

🔧 Creating FAISS index...
✓ FAISS index created with 18717 vectors

💾 Saving to disk...
✓ FAISS index saved: ../../04_models/vector_index/faiss_index.bin
✓ Texts saved: ../../04_models/vector_index/texts.pkl (18717 texts)
✓ Metadata saved: ../../04_models/vector_index/metadata.pkl (18717 entries)
✓ Embeddings saved: ../../04_models/vector_index/embeddings.npy
✓ Index info saved: ../../04_models/vector_index/index_info.json

✅ FAISS INDEX CREATION COMPLETE

📊 STATS:
Total chunks: 18717
Embedding dimension: 384
FAISS index size: 18717 vectors

🧪 TEST QUERY:
Query: 'automotive technology'
Found 3 results:

  Result 1 (distance: 0.6645):
    Type: research_paper
    Preview: (V2X), Internet of Things (IOT), public clouds, data analytics, artificial intelligence, digitalizat...

  Result 2 (distance: 0.7099):
    Type: r

### Retriever prototype

Load and test the retriever

In [ ]:
def import_retriever():
    current_dir = os.getcwd()
    retriever_path = os.path.join(current_dir, 'rag_components', 'retriever.py')
    spec = importlib.util.spec_from_file_location("retriever", retriever_path)
    retriever_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(retriever_module)
    return retriever_module.DocumentAwareRetriever

DocumentAwareRetriever = import_retriever()
print("✅ Retriever imported!")

✅ Retriever imported!


In [ ]:
# Initialize the retriever
print("Initializing TF-IDF retriever...")
VECTOR_INDEX_PATH = "../../04_models/vector_index"
retriever = DocumentAwareRetriever(VECTOR_INDEX_PATH)

# Test the retriever with startups data
print("\n🧪 Testing retriever with automotive-focused queries...")

test_queries = [
    "automotive startups",
    "autonomous driving technology", 
    "generative AI in automotive",
    "electric vehicle innovation"
]

for query in test_queries:
    results = retriever.retrieve_with_sources(query, k=3)
    
    if results:
        print(f"\n🔍 Query: '{query}'")
        print(f"📊 Top result: {results[0]['source_file']}")
        print(f"📝 Type: {results[0]['doc_type']}")
        print(f"⭐ Score: {results[0]['similarity_score']:.4f}")
        
        # Check if startups data was retrieved
        startups_found = any(doc['doc_type'] == 'startups_data' for doc in results)
        print(f"🚀 Startups data included: {'✓' if startups_found else '✗'}")
    else:
        print(f"\n❌ No results for: '{query}'")

# Show document type distribution
doc_type_counts = retriever.get_doc_type_counts()
print(f"\n📈 Document type distribution:")
for doc_type, count in doc_type_counts.items():
    print(f"  - {doc_type}: {count} chunks")

Document type imbalance:

Research papers: 12,466 chunks (72%)
Patents: 4,198 chunks (24%)
Startups: 1,366 chunks (4%) ← Only 4% of your data!

Updated code using FAISS

In [22]:
# SIMPLIFIED DIRECT TEST (no import needed)

print("🧪 Direct FAISS index test from Notebook 2")
print("="*60)

import os
import pickle
import numpy as np
import faiss
from fastembed import TextEmbedding

# Path to your FAISS index
VECTOR_INDEX_PATH = "../../04_models/vector_index"

print(f"Index path: {VECTOR_INDEX_PATH}")
print(f"Path exists: {os.path.exists(VECTOR_INDEX_PATH)}")

if os.path.exists(VECTOR_INDEX_PATH):
    print(f"\n📁 Directory contents:")
    for file in os.listdir(VECTOR_INDEX_PATH):
        file_path = os.path.join(VECTOR_INDEX_PATH, file)
        size = os.path.getsize(file_path)
        print(f"  - {file} ({size:,} bytes)")
    
    try:
        # 1. Load FAISS index
        index_path = os.path.join(VECTOR_INDEX_PATH, "faiss_index.bin")
        index = faiss.read_index(index_path)
        print(f"\n✅ FAISS index loaded: {index.ntotal} vectors")
        
        # 2. Load texts
        texts_path = os.path.join(VECTOR_INDEX_PATH, "texts.pkl")
        with open(texts_path, "rb") as f:
            texts = pickle.load(f)
        print(f"✅ Texts loaded: {len(texts)} chunks")
        
        # 3. Load metadata
        metadata_path = os.path.join(VECTOR_INDEX_PATH, "metadata.pkl")
        with open(metadata_path, "rb") as f:
            metadatas = pickle.load(f)
        print(f"✅ Metadata loaded: {len(metadatas)} entries")
        
        # 4. Initialize embedding model for queries
        model = TextEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
        print("✅ Embedding model loaded")
        
        # 5. Test queries
        test_queries = [
            "automotive startups",
            "autonomous driving technology", 
            "generative AI in automotive",
            "electric vehicle innovation"
        ]
        
        print(f"\n{'='*60}")
        print("🧪 Testing FAISS search directly:")
        
        for query in test_queries:
            print(f"\n🔍 Query: '{query}'")
            
            # Create query embedding
            query_embedding = np.array(
                list(model.embed([query]))[0].tolist()
            ).astype('float32').reshape(1, -1)
            
            # Search
            k = 3
            distances, indices = index.search(query_embedding, k)
            
            # Show results
            found_results = 0
            for i, (idx, distance) in enumerate(zip(indices[0], distances[0])):
                if idx != -1 and idx < len(texts):
                    similarity = 1.0 / (1.0 + distance)
                    if similarity > 0.5:  # Threshold
                        found_results += 1
                        doc_type = metadatas[idx].get('doc_type', 'N/A') if idx < len(metadatas) else 'N/A'
                        source = metadatas[idx].get('source', 'N/A') if idx < len(metadatas) else 'N/A'
                        
                        print(f"  Result {i+1}:")
                        print(f"    Type: {doc_type}")
                        print(f"    Source: {source}")
                        print(f"    Similarity: {similarity:.4f}")
                        
                        # Check if startups
                        if doc_type == 'startups':
                            print(f"    🚀 Startups data: ✓")
            
            if found_results == 0:
                print(f"  ❌ No results above threshold 0.5")
        
        # Show document distribution
        print(f"\n{'='*60}")
        print("📈 Document type distribution:")
        doc_type_counts = {}
        for metadata in metadatas:
            doc_type = metadata.get('doc_type', 'unknown')
            doc_type_counts[doc_type] = doc_type_counts.get(doc_type, 0) + 1
        
        for doc_type, count in sorted(doc_type_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"  - {doc_type}: {count} chunks")
        
        print(f"\n{'='*60}")
        print("✅ Direct FAISS test completed successfully!")
        
    except Exception as e:
        print(f"❌ Error during direct test: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ Vector index directory not found!")
    print("Make sure you ran the FAISS embedding creation code first.")

🧪 Direct FAISS index test from Notebook 2
Index path: ../../04_models/vector_index
Path exists: True

📁 Directory contents:
  - metadata.pkl (431,988 bytes)
  - faiss_index.bin (28,749,357 bytes)
  - embeddings.npy (28,749,440 bytes)
  - index_info.json (187 bytes)
  - texts.pkl (11,096,569 bytes)

✅ FAISS index loaded: 18717 vectors
✅ Texts loaded: 18717 chunks
✅ Metadata loaded: 18717 entries
✅ Embedding model loaded

🧪 Testing FAISS search directly:

🔍 Query: 'automotive startups'
  Result 1:
    Type: research_paper
    Source: automotive_papers_processed.txt
    Similarity: 0.5672
  Result 2:
    Type: startups
    Source: autotechinsight_startups_processed.txt
    Similarity: 0.5527
    🚀 Startups data: ✓
  Result 3:
    Type: startups
    Source: autotechinsight_startups_processed.txt
    Similarity: 0.5459
    🚀 Startups data: ✓

🔍 Query: 'autonomous driving technology'
  Result 1:
    Type: research_paper
    Source: automotive_papers_processed.txt
    Similarity: 0.6073
  Res